In [1]:
from nltk.tokenize import sent_tokenize, word_tokenize
import warnings

In [2]:
warnings.filterwarnings(action = 'ignore')

In [38]:
import gensim
from gensim.models import word2vec
from gensim.models.word2vec import Word2Vec
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import spacy
import string


2023-04-30 12:14:22.594792: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-30 12:14:22.835168: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-30 12:14:22.836795: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-30 12:14:24.159077: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-04-30 12:14:25.658628: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Do

In [36]:
pip install -U spacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 996.8 kB/s eta 0:00:00eta 0:00:010:00:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.7/128.7 kB 388.4 kB/s eta 0:00:00 kB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 805.0 kB/s eta 0:00:00m eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.6/181.6 kB 661.9 kB/s eta 0:00:00 kB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 661.4 kB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.6/491.6 kB 468.6 kB/s eta 0:00:001m461.5 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 922.5/922.5 kB 514.5 kB/s eta 0:00:000:00:01m eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 241.3 kB/s eta 0:00:00m eta 0:00:010:00:02
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd

In [5]:
dev=pd.read_json('dev/EXIST2023_dev.json')

In [6]:
dev_Yes=pd.read_json('dev/EXIST2023_dev.json')
dev_Yes=dev_Yes.transpose()
dev_Yes=dev_Yes.loc[dev_Yes['lang'].apply(lambda x:x=='en')]
dev_Yes=dev_Yes.loc[dev_Yes['labels_task1'].apply(lambda x:x.count('YES')>3)]
dev_Yes['label1']='YES'
dev_Yes

,id_EXIST,lang,tweet,number_annotators,annotators,gender_annotators,age_annotators,labels_task1,labels_task2,labels_task3,split,label1
400002,400002,en,@BBCWomansHour @LabWomenDec @EverydaySexism Sh...,6,"[Annotator_770, Annotator_771, Annotator_772, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[YES, YES, NO, YES, YES, NO]","[REPORTED, JUDGEMENTAL, -, REPORTED, REPORTED, -]","[[IDEOLOGICAL-INEQUALITY], [OBJECTIFICATION], ...",DEV_EN,YES
400003,400003,en,#everydaysexism Some man moving my suitcase in...,6,"[Annotator_776, Annotator_777, Annotator_195, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, YES, YES, YES, YES, YES]","[-, REPORTED, REPORTED, REPORTED, REPORTED, JU...","[[-], [STEREOTYPING-DOMINANCE], [OBJECTIFICATI...",DEV_EN,YES
400008,400008,en,@ReproRights @AbortionStories Getting Twitter ...,6,"[Annotator_791, Annotator_122, Annotator_396, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, YES, NO, YES, YES, YES]","[-, REPORTED, -, DIRECT, JUDGEMENTAL, DIRECT]","[[-], [OBJECTIFICATION, SEXUAL-VIOLENCE], [-],...",DEV_EN,YES
400011,400011,en,@esjayXX @EcuadorianMum @monsalore They so rem...,6,"[Annotator_795, Annotator_796, Annotator_797, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, YES, YES, YES, NO, YES]","[-, REPORTED, REPORTED, DIRECT, -, JUDGEMENTAL]","[[-], [OBJECTIFICATION], [MISOGYNY-NON-SEXUAL-...",DEV_EN,YES
400013,400013,en,One of the depressing things about #NotAllMen ...,6,"[Annotator_764, Annotator_765, Annotator_766, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, YES, NO, YES, YES, YES]","[-, DIRECT, -, DIRECT, JUDGEMENTAL, JUDGEMENTAL]","[[-], [STEREOTYPING-DOMINANCE], [-], [IDEOLOGI...",DEV_EN,YES
...,...,...,...,...,...,...,...,...,...,...,...,...
400485,400485,en,@YesReallyAngel “Don’t wear a black bra with a...,6,"[Annotator_801, Annotator_182, Annotator_802, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[YES, YES, YES, YES, YES, YES]","[DIRECT, DIRECT, JUDGEMENTAL, DIRECT, DIRECT, ...","[[STEREOTYPING-DOMINANCE, OBJECTIFICATION, SEX...",DEV_EN,YES
400486,400486,en,""" get changed , you look like a prostitute . ""...",6,"[Annotator_801, Annotator_182, Annotator_802, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[YES, YES, YES, YES, NO, YES]","[DIRECT, REPORTED, REPORTED, REPORTED, -, DIRECT]","[[IDEOLOGICAL-INEQUALITY, STEREOTYPING-DOMINAN...",DEV_EN,YES
400487,400487,en,made this top and my mom gave me the “you look...,6,"[Annotator_795, Annotator_796, Annotator_797, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[YES, YES, YES, YES, YES, NO]","[JUDGEMENTAL, REPORTED, REPORTED, REPORTED, RE...","[[OBJECTIFICATION], [OBJECTIFICATION], [OBJECT...",DEV_EN,YES
400488,400488,en,@DawnAnd91320913 I haven't seen anything that ...,6,"[Annotator_776, Annotator_777, Annotator_195, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[YES, YES, YES, YES, YES, YES]","[DIRECT, DIRECT, DIRECT, DIRECT, DIRECT, DIRECT]","[[IDEOLOGICAL-INEQUALITY, STEREOTYPING-DOMINAN...",DEV_EN,YES


In [7]:
dev_No=pd.read_json('dev/EXIST2023_dev.json')
dev_No=dev_No.transpose()
dev_No=dev_No.loc[dev_No['lang'].apply(lambda x:x=='en')]
dev_No=dev_No.loc[dev_No['labels_task1'].apply(lambda x:x.count('NO')>3)]
dev_No['label1']='NO'
dev_No

,id_EXIST,lang,tweet,number_annotators,annotators,gender_annotators,age_annotators,labels_task1,labels_task2,labels_task3,split,label1
400001,400001,en,"@Mike_Fabricant “You should smile more, love. ...",6,"[Annotator_764, Annotator_765, Annotator_766, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, NO, NO, NO, YES, YES]","[-, -, -, -, REPORTED, DIRECT]","[[-], [-], [-], [-], [IDEOLOGICAL-INEQUALITY, ...",DEV_EN,NO
400004,400004,en,@KolHue @OliverJia1014 lol gamergate the go to...,6,"[Annotator_780, Annotator_781, Annotator_782, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[YES, NO, NO, NO, NO, NO]","[DIRECT, -, -, -, -, -]","[[STEREOTYPING-DOMINANCE, OBJECTIFICATION, MIS...",DEV_EN,NO
400005,400005,en,@ShelfStoriesGBL To me this has the same negat...,6,"[Annotator_780, Annotator_781, Annotator_782, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[YES, NO, NO, YES, NO, NO]","[JUDGEMENTAL, -, -, REPORTED, -, -]","[[IDEOLOGICAL-INEQUALITY], [-], [-], [MISOGYNY...",DEV_EN,NO
400006,400006,en,@IrrelevantCmnt @jbo911 @BanButterfly @TheRigh...,6,"[Annotator_780, Annotator_781, Annotator_782, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[YES, NO, NO, NO, NO, NO]","[DIRECT, -, -, -, -, -]","[[STEREOTYPING-DOMINANCE, MISOGYNY-NON-SEXUAL-...",DEV_EN,NO
400007,400007,en,@cathymwafer @andrew_lilico Showing off? The m...,6,"[Annotator_785, Annotator_786, Annotator_787, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, NO, NO, NO, NO, NO]","[-, -, -, -, -, -]","[[-], [-], [-], [-], [-], [-]]",DEV_EN,NO
...,...,...,...,...,...,...,...,...,...,...,...,...
400474,400474,en,#cryptosterone 👈All-new!Twice as many #men inv...,6,"[Annotator_770, Annotator_771, Annotator_772, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, NO, NO, YES, NO, NO]","[-, -, -, DIRECT, -, -]","[[-], [-], [-], [IDEOLOGICAL-INEQUALITY, STERE...",DEV_EN,NO
400476,400476,en,@ScubaMySteve If you ask Cenk as a hypothetica...,6,"[Annotator_805, Annotator_426, Annotator_806, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, YES, NO, NO, NO, NO]","[-, JUDGEMENTAL, -, -, -, -]","[[-], [IDEOLOGICAL-INEQUALITY, MISOGYNY-NON-SE...",DEV_EN,NO
400477,400477,en,@snowflowergomi @minseulsoo @chimsmaze How wil...,6,"[Annotator_795, Annotator_796, Annotator_797, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, NO, NO, NO, NO, YES]","[-, -, -, -, -, REPORTED]","[[-], [-], [-], [-], [-], [STEREOTYPING-DOMINA...",DEV_EN,NO
400481,400481,en,"Uh, they're allowed to? calling this out makes...",6,"[Annotator_805, Annotator_426, Annotator_806, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, NO, YES, NO, YES, NO]","[-, -, JUDGEMENTAL, -, DIRECT, -]","[[-], [-], [MISOGYNY-NON-SEXUAL-VIOLENCE], [-]...",DEV_EN,NO


In [8]:
devs = [dev_Yes, dev_No]
dev = pd.concat(devs)
dev.reset_index(drop=True)

,id_EXIST,lang,tweet,number_annotators,annotators,gender_annotators,age_annotators,labels_task1,labels_task2,labels_task3,split,label1
0,400002,en,@BBCWomansHour @LabWomenDec @EverydaySexism Sh...,6,"[Annotator_770, Annotator_771, Annotator_772, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[YES, YES, NO, YES, YES, NO]","[REPORTED, JUDGEMENTAL, -, REPORTED, REPORTED, -]","[[IDEOLOGICAL-INEQUALITY], [OBJECTIFICATION], ...",DEV_EN,YES
1,400003,en,#everydaysexism Some man moving my suitcase in...,6,"[Annotator_776, Annotator_777, Annotator_195, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, YES, YES, YES, YES, YES]","[-, REPORTED, REPORTED, REPORTED, REPORTED, JU...","[[-], [STEREOTYPING-DOMINANCE], [OBJECTIFICATI...",DEV_EN,YES
2,400008,en,@ReproRights @AbortionStories Getting Twitter ...,6,"[Annotator_791, Annotator_122, Annotator_396, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, YES, NO, YES, YES, YES]","[-, REPORTED, -, DIRECT, JUDGEMENTAL, DIRECT]","[[-], [OBJECTIFICATION, SEXUAL-VIOLENCE], [-],...",DEV_EN,YES
3,400011,en,@esjayXX @EcuadorianMum @monsalore They so rem...,6,"[Annotator_795, Annotator_796, Annotator_797, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, YES, YES, YES, NO, YES]","[-, REPORTED, REPORTED, DIRECT, -, JUDGEMENTAL]","[[-], [OBJECTIFICATION], [MISOGYNY-NON-SEXUAL-...",DEV_EN,YES
4,400013,en,One of the depressing things about #NotAllMen ...,6,"[Annotator_764, Annotator_765, Annotator_766, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, YES, NO, YES, YES, YES]","[-, DIRECT, -, DIRECT, JUDGEMENTAL, JUDGEMENTAL]","[[-], [STEREOTYPING-DOMINANCE], [-], [IDEOLOGI...",DEV_EN,YES
...,...,...,...,...,...,...,...,...,...,...,...,...
439,400474,en,#cryptosterone 👈All-new!Twice as many #men inv...,6,"[Annotator_770, Annotator_771, Annotator_772, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, NO, NO, YES, NO, NO]","[-, -, -, DIRECT, -, -]","[[-], [-], [-], [IDEOLOGICAL-INEQUALITY, STERE...",DEV_EN,NO
440,400476,en,@ScubaMySteve If you ask Cenk as a hypothetica...,6,"[Annotator_805, Annotator_426, Annotator_806, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, YES, NO, NO, NO, NO]","[-, JUDGEMENTAL, -, -, -, -]","[[-], [IDEOLOGICAL-INEQUALITY, MISOGYNY-NON-SE...",DEV_EN,NO
441,400477,en,@snowflowergomi @minseulsoo @chimsmaze How wil...,6,"[Annotator_795, Annotator_796, Annotator_797, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, NO, NO, NO, NO, YES]","[-, -, -, -, -, REPORTED]","[[-], [-], [-], [-], [-], [STEREOTYPING-DOMINA...",DEV_EN,NO
442,400481,en,"Uh, they're allowed to? calling this out makes...",6,"[Annotator_805, Annotator_426, Annotator_806, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, NO, YES, NO, YES, NO]","[-, -, JUDGEMENTAL, -, DIRECT, -]","[[-], [-], [MISOGYNY-NON-SEXUAL-VIOLENCE], [-]...",DEV_EN,NO


In [9]:
train_Yes=pd.read_json('training/EXIST2023_training.json')
train_Yes=train_Yes.transpose()
train_Yes=train_Yes.loc[train_Yes['lang'].apply(lambda x:x=='en')]
train_Yes=train_Yes.loc[train_Yes['labels_task1'].apply(lambda x:x.count('YES')>3)]
train_Yes['label1']='YES'
train_Yes

,id_EXIST,lang,tweet,number_annotators,annotators,gender_annotators,age_annotators,labels_task1,labels_task2,labels_task3,split,label1
200002,200002,en,Writing a uni essay in my local pub with a cof...,6,"[Annotator_391, Annotator_392, Annotator_393, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[YES, YES, YES, NO, YES, YES]","[REPORTED, DIRECT, REPORTED, -, REPORTED, REPO...","[[STEREOTYPING-DOMINANCE, OBJECTIFICATION, MIS...",TRAIN_EN,YES
200003,200003,en,@UniversalORL it is 2021 not 1921. I dont appr...,6,"[Annotator_397, Annotator_398, Annotator_399, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[YES, YES, NO, YES, NO, YES]","[REPORTED, REPORTED, -, REPORTED, -, JUDGEMENTAL]","[[OBJECTIFICATION, SEXUAL-VIOLENCE], [STEREOTY...",TRAIN_EN,YES
200006,200006,en,According to a customer I have plenty of time ...,6,"[Annotator_409, Annotator_410, Annotator_411, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[YES, YES, YES, YES, YES, YES]","[REPORTED, REPORTED, REPORTED, REPORTED, REPOR...","[[STEREOTYPING-DOMINANCE, OBJECTIFICATION], [S...",TRAIN_EN,YES
200007,200007,en,"So only 'blokes' drink beer? Sorry, but if you...",6,"[Annotator_415, Annotator_416, Annotator_417, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[YES, YES, YES, YES, YES, YES]","[JUDGEMENTAL, REPORTED, REPORTED, DIRECT, DIRE...","[[STEREOTYPING-DOMINANCE], [STEREOTYPING-DOMIN...",TRAIN_EN,YES
200011,200011,en,#EverydaySexism means women usually end up in ...,6,"[Annotator_437, Annotator_438, Annotator_439, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[YES, YES, YES, YES, YES, YES]","[REPORTED, JUDGEMENTAL, JUDGEMENTAL, JUDGEMENT...","[[IDEOLOGICAL-INEQUALITY, STEREOTYPING-DOMINAN...",TRAIN_EN,YES
...,...,...,...,...,...,...,...,...,...,...,...,...
203256,203256,en,idk why y’all bitches think having half your a...,6,"[Annotator_478, Annotator_479, Annotator_480, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[YES, YES, YES, YES, YES, YES]","[JUDGEMENTAL, DIRECT, DIRECT, DIRECT, JUDGEMEN...","[[OBJECTIFICATION], [STEREOTYPING-DOMINANCE, S...",TRAIN_EN,YES
203257,203257,en,This has been a part of an experiment with @Wo...,6,"[Annotator_668, Annotator_669, Annotator_670, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[YES, YES, YES, YES, YES, YES]","[JUDGEMENTAL, REPORTED, JUDGEMENTAL, DIRECT, J...","[[OBJECTIFICATION], [OBJECTIFICATION], [OBJECT...",TRAIN_EN,YES
203258,203258,en,"""Take me already"" ""Not yet. You gotta be ready...",6,"[Annotator_467, Annotator_468, Annotator_469, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[NO, YES, NO, YES, YES, YES]","[-, DIRECT, -, DIRECT, DIRECT, JUDGEMENTAL]","[[-], [OBJECTIFICATION], [-], [SEXUAL-VIOLENCE...",TRAIN_EN,YES
203259,203259,en,@clintneedcoffee why do you look like a whore?...,6,"[Annotator_674, Annotator_675, Annotator_676, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[YES, YES, YES, YES, YES, YES]","[DIRECT, DIRECT, DIRECT, DIRECT, JUDGEMENTAL, ...","[[OBJECTIFICATION, SEXUAL-VIOLENCE, MISOGYNY-N...",TRAIN_EN,YES


In [10]:
train_No=pd.read_json('training/EXIST2023_training.json')
train_No=train_No.transpose()
train_No=train_No.loc[train_No['lang'].apply(lambda x:x=='en')]
train_No=train_No.loc[train_No['labels_task1'].apply(lambda x:x.count('NO')>3)]
train_No['label1']='NO'
train_No

,id_EXIST,lang,tweet,number_annotators,annotators,gender_annotators,age_annotators,labels_task1,labels_task2,labels_task3,split,label1
200008,200008,en,New to the shelves this week - looking forward...,6,"[Annotator_420, Annotator_296, Annotator_421, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[NO, NO, NO, YES, NO, NO]","[-, -, -, JUDGEMENTAL, -, -]","[[-], [-], [-], [IDEOLOGICAL-INEQUALITY], [-],...",TRAIN_EN,NO
200010,200010,en,I guess that’s fairly normal for a Neanderthal...,6,"[Annotator_431, Annotator_432, Annotator_433, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[NO, NO, NO, NO, YES, NO]","[-, -, -, -, JUDGEMENTAL, -]","[[-], [-], [-], [-], [OBJECTIFICATION, SEXUAL-...",TRAIN_EN,NO
200023,200023,en,@JavedLSterritt He's really shit on games jour...,6,"[Annotator_496, Annotator_497, Annotator_498, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[NO, NO, NO, YES, NO, NO]","[-, -, -, JUDGEMENTAL, -, -]","[[-], [-], [-], [IDEOLOGICAL-INEQUALITY, STERE...",TRAIN_EN,NO
200024,200024,en,"Low key, she was fr created because the Incred...",6,"[Annotator_502, Annotator_503, Annotator_504, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[NO, NO, NO, NO, NO, YES]","[-, -, -, -, -, JUDGEMENTAL]","[[-], [-], [-], [-], [-], [STEREOTYPING-DOMINA...",TRAIN_EN,NO
200025,200025,en,"One of the most important (and of course, unde...",6,"[Annotator_508, Annotator_509, Annotator_510, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[NO, NO, NO, NO, NO, NO]","[-, -, -, -, -, -]","[[-], [-], [-], [-], [-], [-]]",TRAIN_EN,NO
...,...,...,...,...,...,...,...,...,...,...,...,...
203180,203180,en,"@BFriedmanDC ""The other officers yell after Jo...",6,"[Annotator_617, Annotator_618, Annotator_619, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[NO, YES, NO, NO, NO, NO]","[-, REPORTED, -, -, -, -]","[[-], [STEREOTYPING-DOMINANCE], [-], [-], [-],...",TRAIN_EN,NO
203201,203201,en,@v1sig0th @PTSDGuts2 Get off your goddamn high...,6,"[Annotator_508, Annotator_509, Annotator_510, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[NO, YES, NO, NO, YES, NO]","[-, DIRECT, -, -, DIRECT, -]","[[-], [SEXUAL-VIOLENCE], [-], [-], [STEREOTYPI...",TRAIN_EN,NO
203202,203202,en,I hate the phrase “when you assume it makes an...,6,"[Annotator_703, Annotator_704, Annotator_705, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[NO, YES, NO, YES, NO, NO]","[-, DIRECT, -, DIRECT, -, -]","[[-], [OBJECTIFICATION], [-], [STEREOTYPING-DO...",TRAIN_EN,NO
203205,203205,en,To the guy who beeped at me after I'd put my h...,6,"[Annotator_645, Annotator_646, Annotator_647, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[YES, NO, NO, NO, YES, NO]","[REPORTED, -, -, -, DIRECT, -]","[[STEREOTYPING-DOMINANCE], [-], [-], [-], [STE...",TRAIN_EN,NO


In [11]:
frames = [train_Yes, train_No]
result = pd.concat(frames)
result.reset_index(drop=True)

,id_EXIST,lang,tweet,number_annotators,annotators,gender_annotators,age_annotators,labels_task1,labels_task2,labels_task3,split,label1
0,200002,en,Writing a uni essay in my local pub with a cof...,6,"[Annotator_391, Annotator_392, Annotator_393, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[YES, YES, YES, NO, YES, YES]","[REPORTED, DIRECT, REPORTED, -, REPORTED, REPO...","[[STEREOTYPING-DOMINANCE, OBJECTIFICATION, MIS...",TRAIN_EN,YES
1,200003,en,@UniversalORL it is 2021 not 1921. I dont appr...,6,"[Annotator_397, Annotator_398, Annotator_399, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[YES, YES, NO, YES, NO, YES]","[REPORTED, REPORTED, -, REPORTED, -, JUDGEMENTAL]","[[OBJECTIFICATION, SEXUAL-VIOLENCE], [STEREOTY...",TRAIN_EN,YES
2,200006,en,According to a customer I have plenty of time ...,6,"[Annotator_409, Annotator_410, Annotator_411, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[YES, YES, YES, YES, YES, YES]","[REPORTED, REPORTED, REPORTED, REPORTED, REPOR...","[[STEREOTYPING-DOMINANCE, OBJECTIFICATION], [S...",TRAIN_EN,YES
3,200007,en,"So only 'blokes' drink beer? Sorry, but if you...",6,"[Annotator_415, Annotator_416, Annotator_417, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[YES, YES, YES, YES, YES, YES]","[JUDGEMENTAL, REPORTED, REPORTED, DIRECT, DIRE...","[[STEREOTYPING-DOMINANCE], [STEREOTYPING-DOMIN...",TRAIN_EN,YES
4,200011,en,#EverydaySexism means women usually end up in ...,6,"[Annotator_437, Annotator_438, Annotator_439, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[YES, YES, YES, YES, YES, YES]","[REPORTED, JUDGEMENTAL, JUDGEMENTAL, JUDGEMENT...","[[IDEOLOGICAL-INEQUALITY, STEREOTYPING-DOMINAN...",TRAIN_EN,YES
...,...,...,...,...,...,...,...,...,...,...,...,...
2865,203180,en,"@BFriedmanDC ""The other officers yell after Jo...",6,"[Annotator_617, Annotator_618, Annotator_619, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[NO, YES, NO, NO, NO, NO]","[-, REPORTED, -, -, -, -]","[[-], [STEREOTYPING-DOMINANCE], [-], [-], [-],...",TRAIN_EN,NO
2866,203201,en,@v1sig0th @PTSDGuts2 Get off your goddamn high...,6,"[Annotator_508, Annotator_509, Annotator_510, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[NO, YES, NO, NO, YES, NO]","[-, DIRECT, -, -, DIRECT, -]","[[-], [SEXUAL-VIOLENCE], [-], [-], [STEREOTYPI...",TRAIN_EN,NO
2867,203202,en,I hate the phrase “when you assume it makes an...,6,"[Annotator_703, Annotator_704, Annotator_705, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[NO, YES, NO, YES, NO, NO]","[-, DIRECT, -, DIRECT, -, -]","[[-], [OBJECTIFICATION], [-], [STEREOTYPING-DO...",TRAIN_EN,NO
2868,203205,en,To the guy who beeped at me after I'd put my h...,6,"[Annotator_645, Annotator_646, Annotator_647, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[YES, NO, NO, NO, YES, NO]","[REPORTED, -, -, -, DIRECT, -]","[[STEREOTYPING-DOMINANCE], [-], [-], [-], [STE...",TRAIN_EN,NO


In [12]:
!pip install --upgrade gensim -q
!pip install matplotlib -q

In [13]:
import gensim
from gensim.models import word2vec
from gensim.models.word2vec import Word2Vec
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import string

In [14]:
gensim.__version__

'4.3.1'

In [15]:
import gensim.downloader as api
print(list(gensim.downloader.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [16]:
wv = api.load('word2vec-google-news-300')

In [17]:
wv.save('training/vector.kv')

In [18]:
type(wv)

gensim.models.keyedvectors.KeyedVectors

In [19]:
def sent_vec(sent):
    vector_size = wv.vector_size
    wv_res = np.zeros(vector_size)
    ctr = 1
    for w in sent:
        if w in wv:
            ctr += 1
            wv_res += wv[w]
    wv_res = wv_res/ctr
    return wv_res

In [20]:
import preprocessor as p

In [21]:
def preprocess_tweet(row):
    text = row['tweet']
    text = p.clean(text)
    return text

In [124]:
result['tweet'] = result.apply(preprocess_tweet, axis=1)
dev['tweet'] = dev.apply(preprocess_tweet, axis=1)

In [23]:
from gensim.parsing.preprocessing import remove_stopwords

In [24]:
def stopword_removal(row):
    text = row['tweet']
    text = remove_stopwords(text)
    
    return text

In [125]:
import re
result['tweet'] = result.apply(stopword_removal, axis=1)
result.tweet = result.tweet.apply(lambda x: re.sub(r'https?:\/\/\S+', '', x))
result.tweet.apply(lambda x: re.sub(r"www\.[a-z]?\.?(com)+|[a-z]+\.(com)", '', x))
result.tweet = result.tweet.apply(lambda x: re.sub(r'&[a-z]+;', '', x))
result.tweet = result.tweet.apply(lambda x: re.sub(r"[^a-z\s\(\-:\)\\\/\];='#]", '', x))
result.tweet = result.tweet.apply(lambda x: re.sub(r'@mention', '', x))
dev['tweet'] = dev.apply(stopword_removal, axis=1)
dev.tweet = dev.tweet.apply(lambda x: re.sub(r'https?:\/\/\S+', '', x))
dev.tweet.apply(lambda x: re.sub(r"www\.[a-z]?\.?(com)+|[a-z]+\.(com)", '', x))
dev.tweet = dev.tweet.apply(lambda x: re.sub(r'&[a-z]+;', '', x))
dev.tweet = dev.tweet.apply(lambda x: re.sub(r"[^a-z\s\(\-:\)\\\/\];='#]", '', x))
dev.tweet = dev.tweet.apply(lambda x: re.sub(r'@mention', '', x))

In [126]:
result.head()

,id_EXIST,lang,tweet,number_annotators,annotators,gender_annotators,age_annotators,labels_task1,labels_task2,labels_task3,split,label1,tokens,vec
200002,200002,en,writing uni essay local pub coffee random old ...,6,"[Annotator_391, Annotator_392, Annotator_393, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[YES, YES, YES, NO, YES, YES]","[REPORTED, DIRECT, REPORTED, -, REPORTED, REPO...","[[STEREOTYPING-DOMINANCE, OBJECTIFICATION, MIS...",TRAIN_EN,YES,"[writing, uni, essay, local, pub, coffee, rand...","[0.04673062838040865, 0.018944373497596152, -0..."
200003,200003,en,dont appreciate rides team member looked asked...,6,"[Annotator_397, Annotator_398, Annotator_399, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[YES, YES, NO, YES, NO, YES]","[REPORTED, REPORTED, -, REPORTED, -, JUDGEMENTAL]","[[OBJECTIFICATION, SEXUAL-VIOLENCE], [STEREOTY...",TRAIN_EN,YES,"[i, dont, appreciate, rides, team, member, loo...","[0.001953125, 0.044283353365384616, 0.04774357..."
200006,200006,en,according customer plenty time spent stirling ...,6,"[Annotator_409, Annotator_410, Annotator_411, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[YES, YES, YES, YES, YES, YES]","[REPORTED, REPORTED, REPORTED, REPORTED, REPOR...","[[STEREOTYPING-DOMINANCE, OBJECTIFICATION], [S...",TRAIN_EN,YES,"[according, customer, i, plenty, time, spent, ...","[0.05580540707236842, 0.043840809872275906, -0..."
200007,200007,en,'blokes' drink beer sorry aren't 'bloke' drink...,6,"[Annotator_415, Annotator_416, Annotator_417, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[YES, YES, YES, YES, YES, YES]","[JUDGEMENTAL, REPORTED, REPORTED, DIRECT, DIRE...","[[STEREOTYPING-DOMINANCE], [STEREOTYPING-DOMIN...",TRAIN_EN,YES,"[so, 'blokes', drink, beer, sorry, aren't, 'bl...","[-0.06791229248046875, -0.06807861328125, 0.01..."
200011,200011,en,means women usually end lower paid support wor...,6,"[Annotator_437, Annotator_438, Annotator_439, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[YES, YES, YES, YES, YES, YES]","[REPORTED, JUDGEMENTAL, JUDGEMENTAL, JUDGEMENT...","[[IDEOLOGICAL-INEQUALITY, STEREOTYPING-DOMINAN...",TRAIN_EN,YES,"[means, women, usually, end, lower, paid, supp...","[-0.008846507352941176, 0.019739487591911766, ..."


In [127]:
len(wv['apple'])

300

In [128]:
pairs = [
    ('car', 'minivan'),   # a minivan is a kind of car
    ('car', 'bicycle'),   # still a wheeled vehicle
    ('car', 'airplane'),  # ok, no wheels, but still a vehicle
    ('car', 'cereal'),    # ... and so on
    ('car', 'communism'),
]
for w1, w2 in pairs:
    print('%r\t%r\t%.2f' % (w1, w2, wv.similarity(w1, w2)))

'car'	'minivan'	0.69
'car'	'bicycle'	0.54
'car'	'airplane'	0.42
'car'	'cereal'	0.14
'car'	'communism'	0.06


In [129]:
vec=wv['Germany']-wv['Berlin']+wv['Paris']
wv.most_similar(vec)

[('France', 0.7724406123161316),
 ('Paris', 0.6798243522644043),
 ('Belgium', 0.598486065864563),
 ('Germany', 0.5652832388877869),
 ('extradites_Noriega', 0.5623601078987122),
 ('Villebon_Sur_Yvette', 0.5570637583732605),
 ('Spain', 0.550815761089325),
 ('Italy', 0.5462924838066101),
 ('Marseille', 0.5372346639633179),
 ('Switzerland', 0.5364957451820374)]

In [28]:
def sent_vec(sent):
    vector_size = wv.vector_size
    wv_res = np.zeros(vector_size)
    ctr = 1
    for w in sent:
        if w in wv:
            ctr += 1
            wv_res += wv[w]
    wv_res = wv_res/ctr
    return wv_res

In [61]:
def tokenizer(sentence):
   # Creating our token object, which is used to create documents with linguistic annotations
    my=sentence.split()
    mytokens = [word for word in my]
    # return preprocessed list of tokens
    return mytokens

In [59]:
sent_vec("I am happy")

array([-0.08447266,  0.10973685, -0.00453404,  0.18362863, -0.05944824,
       -0.04013497, -0.10041155,  0.03710938, -0.02503313,  0.04729353,
       -0.07543836, -0.16434152, -0.1116333 ,  0.02915737, -0.12290737,
        0.10421317, -0.00425502,  0.27535575, -0.01681083, -0.08075387,
       -0.26747349, -0.07281712,  0.08138602, -0.02498954, -0.05358887,
       -0.07969884, -0.30385045,  0.10531616, -0.03756278, -0.04029192,
        0.06455776, -0.02200753, -0.11361694, -0.12702288, -0.19824219,
        0.06921387, -0.14557757,  0.18457031, -0.05542864,  0.10093471,
        0.10728237, -0.07920619, -0.01236398,  0.11485073,  0.09437779,
        0.02020509, -0.08954729, -0.12911551, -0.0363072 ,  0.05212402,
       -0.12200056,  0.23856027, -0.04056222,  0.12512207,  0.05569894,
        0.1237793 , -0.08370536, -0.05489676, -0.0291748 , -0.18401228,
       -0.14592634, -0.04910714, -0.15844727, -0.03636387,  0.00628662,
       -0.30032785, -0.11404855,  0.08530971,  0.01077706,  0.05

In [68]:
result['tokens'] = result['tweet'].apply(tokenizer)
dev['tokens'] = dev['tweet'].apply(tokenizer)

In [70]:

dev.head()

,id_EXIST,lang,tweet,number_annotators,annotators,gender_annotators,age_annotators,labels_task1,labels_task2,labels_task3,split,label1,tokens
400002,400002,en,she right push opposite direction - converting...,6,"[Annotator_770, Annotator_771, Annotator_772, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[YES, YES, NO, YES, YES, NO]","[REPORTED, JUDGEMENTAL, -, REPORTED, REPORTED, -]","[[IDEOLOGICAL-INEQUALITY], [OBJECTIFICATION], ...",DEV_EN,YES,"[she, right, push, opposite, direction, -, con..."
400003,400003,en,some man moving suitcase overhead luggage stor...,6,"[Annotator_776, Annotator_777, Annotator_195, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, YES, YES, YES, YES, YES]","[-, REPORTED, REPORTED, REPORTED, REPORTED, JU...","[[-], [STEREOTYPING-DOMINANCE], [OBJECTIFICATI...",DEV_EN,YES,"[some, man, moving, suitcase, overhead, luggag..."
400008,400008,en,getting twitter harassed pro choice,6,"[Annotator_791, Annotator_122, Annotator_396, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, YES, NO, YES, YES, YES]","[-, REPORTED, -, DIRECT, JUDGEMENTAL, DIRECT]","[[-], [OBJECTIFICATION, SEXUAL-VIOLENCE], [-],...",DEV_EN,YES,"[getting, twitter, harassed, pro, choice]"
400011,400011,en,they remind mgtow (men way) us men hate women ...,6,"[Annotator_795, Annotator_796, Annotator_797, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, YES, YES, YES, NO, YES]","[-, REPORTED, REPORTED, DIRECT, -, JUDGEMENTAL]","[[-], [OBJECTIFICATION], [MISOGYNY-NON-SEXUAL-...",DEV_EN,YES,"[they, remind, mgtow, (men, way), us, men, hat..."
400013,400013,en,one depressing things primarily rallying privi...,6,"[Annotator_764, Annotator_765, Annotator_766, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, YES, NO, YES, YES, YES]","[-, DIRECT, -, DIRECT, JUDGEMENTAL, JUDGEMENTAL]","[[-], [STEREOTYPING-DOMINANCE], [-], [IDEOLOGI...",DEV_EN,YES,"[one, depressing, things, primarily, rallying,..."


In [72]:
result['vec'] = result['tokens'].apply(sent_vec)
dev['vec'] = dev['tokens'].apply(sent_vec)

In [65]:
result.head()

,id_EXIST,lang,tweet,number_annotators,annotators,gender_annotators,age_annotators,labels_task1,labels_task2,labels_task3,split,label1,tokens,vec
200002,200002,en,writing uni essay local pub coffee random old ...,6,"[Annotator_391, Annotator_392, Annotator_393, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[YES, YES, YES, NO, YES, YES]","[REPORTED, DIRECT, REPORTED, -, REPORTED, REPO...","[[STEREOTYPING-DOMINANCE, OBJECTIFICATION, MIS...",TRAIN_EN,YES,"[writing, uni, essay, local, pub, coffee, rand...","[0.04673062838040865, 0.018944373497596152, -0..."
200003,200003,en,i dont appreciate rides team member looked as...,6,"[Annotator_397, Annotator_398, Annotator_399, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[YES, YES, NO, YES, NO, YES]","[REPORTED, REPORTED, -, REPORTED, -, JUDGEMENTAL]","[[OBJECTIFICATION, SEXUAL-VIOLENCE], [STEREOTY...",TRAIN_EN,YES,"[i, dont, appreciate, rides, team, member, loo...","[0.001953125, 0.044283353365384616, 0.04774357..."
200006,200006,en,according customer i plenty time spent stirlin...,6,"[Annotator_409, Annotator_410, Annotator_411, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[YES, YES, YES, YES, YES, YES]","[REPORTED, REPORTED, REPORTED, REPORTED, REPOR...","[[STEREOTYPING-DOMINANCE, OBJECTIFICATION], [S...",TRAIN_EN,YES,"[according, customer, i, plenty, time, spent, ...","[0.05580540707236842, 0.043840809872275906, -0..."
200007,200007,en,so 'blokes' drink beer sorry aren't 'bloke' dr...,6,"[Annotator_415, Annotator_416, Annotator_417, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[YES, YES, YES, YES, YES, YES]","[JUDGEMENTAL, REPORTED, REPORTED, DIRECT, DIRE...","[[STEREOTYPING-DOMINANCE], [STEREOTYPING-DOMIN...",TRAIN_EN,YES,"[so, 'blokes', drink, beer, sorry, aren't, 'bl...","[-0.06791229248046875, -0.06807861328125, 0.01..."
200011,200011,en,means women usually end lower paid support wor...,6,"[Annotator_437, Annotator_438, Annotator_439, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[YES, YES, YES, YES, YES, YES]","[REPORTED, JUDGEMENTAL, JUDGEMENTAL, JUDGEMENT...","[[IDEOLOGICAL-INEQUALITY, STEREOTYPING-DOMINAN...",TRAIN_EN,YES,"[means, women, usually, end, lower, paid, supp...","[-0.008846507352941176, 0.019739487591911766, ..."


In [130]:
X_train = result['vec'].to_list()
y_train = result['label1'].to_list()
X_test=dev['vec'].to_list()
y_test = dev['label1'].to_list()

In [131]:
y_test = dev.iloc[:, -3].values
y_test =pd.Series(y_test).map({'YES': 1, 'NO': 0})
y_train = result.iloc[:, -3].values
y_train =pd.Series(y_train).map({'YES': 1, 'NO': 0})

In [132]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

In [133]:
scaled = scaler.fit_transform(X_train)
scaled_test = scaler.fit_transform(X_test)

In [134]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(scaled, y_train)
y_pred = classifier.predict(scaled_test)
from sklearn.metrics import accuracy_score, classification_report
classification_report = classification_report(y_test, y_pred)

print('\n Accuracy: ', accuracy_score(y_test, y_pred))
print('\nClassification Report')
print('======================================================')
print('\n', classification_report)


 Accuracy:  0.6351351351351351

Classification Report

               precision    recall  f1-score   support

           0       0.61      0.96      0.75       250
           1       0.82      0.21      0.34       194

    accuracy                           0.64       444
   macro avg       0.72      0.59      0.54       444
weighted avg       0.70      0.64      0.57       444



In [135]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV, StratifiedKFold, learning_curve
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
from sklearn.neural_network import MLPClassifier


In [136]:
kfold = StratifiedKFold( n_splits = 5 )


In [102]:
from sklearn.svm import LinearSVC
sv = LinearSVC(random_state=2018)

param_grid2 = {
    'loss':['squared_hinge'],
    'class_weight':[{1:1},{1:2},{1:3},{1:4},{1:5},{1:6},{1:7}],
    'C': [0.1,0.2,0.3,0.4,0.5,0.6,0.7]
}


gs_sv = GridSearchCV(sv, param_grid = [param_grid2], verbose = 1, cv = kfold, n_jobs = 1, scoring = 'roc_auc')
gs_sv.fit(X_train,y_train)
gs_sv_best = gs_sv.best_estimator_
print(gs_sv.best_params_)


Fitting 5 folds for each of 49 candidates, totalling 245 fits
{'C': 0.1, 'class_weight': {1: 1}, 'loss': 'squared_hinge'}


In [137]:
y_pred = gs_sv.predict(X_test)
from sklearn.metrics import accuracy_score, classification_report
classification_report = classification_report(y_test, y_pred)

print('\n Accuracy: ', accuracy_score(y_test, y_pred))
print('\nClassification Report')
print('======================================================')
print('\n', classification_report)


 Accuracy:  0.7522522522522522

Classification Report

               precision    recall  f1-score   support

           0       0.73      0.88      0.80       250
           1       0.79      0.59      0.68       194

    accuracy                           0.75       444
   macro avg       0.76      0.73      0.74       444
weighted avg       0.76      0.75      0.75       444



In [105]:
from sklearn.naive_bayes import BernoulliNB

bnb = BernoulliNB()
gs_bnb = GridSearchCV(bnb, param_grid = {'alpha': [0.1,0.2,0.3,0.4,0.5,0.6,0.25,0.35],
                                         'binarize': [0.1,0.2,0.15,0.05,0.175,0.125]},  verbose = 1, cv = kfold, n_jobs = 1, scoring = "roc_auc")
gs_bnb.fit(X_train, y_train)
gs_bnb_best = gs_bnb.best_estimator_
print(gs_bnb.best_params_)


Fitting 5 folds for each of 48 candidates, totalling 240 fits
{'alpha': 0.6, 'binarize': 0.05}


In [106]:
y_pred = gs_bnb.predict(X_test)
from sklearn.metrics import accuracy_score, classification_report
classification_report = classification_report(y_test, y_pred)

print('\n Accuracy: ', accuracy_score(y_test, y_pred))
print('\nClassification Report')
print('======================================================')
print('\n', classification_report)


 Accuracy:  0.7004504504504504

Classification Report

               precision    recall  f1-score   support

           0       0.74      0.72      0.73       250
           1       0.65      0.68      0.66       194

    accuracy                           0.70       444
   macro avg       0.70      0.70      0.70       444
weighted avg       0.70      0.70      0.70       444



In [108]:
from xgboost import XGBClassifier

In [109]:
model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

In [110]:
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, predictor=None, random_state=None, ...)

In [111]:
from numpy import nan

In [113]:
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='mlogloss',
              gamma=0, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=16,
              num_parallel_tree=1, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', use_label_encoder=False,
              validate_parameters=1, verbosity=None)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='mlogloss', feature_types=None, gamma=0, gpu_id=-1,
              grow_policy=None, importance_type='gain',
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=0, max_depth=6, max_leaves=None,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=16, num_parallel_tree=1,
              objective='multi:softprob', predictor=None, ...)

In [114]:
y_pred = model.predict(X_test)
from sklearn.metrics import accuracy_score, classification_report
classification_report = classification_report(y_test, y_pred)

print('\n Accuracy: ', accuracy_score(y_test, y_pred))
print('\nClassification Report')
print('======================================================')
print('\n', classification_report)


 Accuracy:  0.7454954954954955

Classification Report

               precision    recall  f1-score   support

           0       0.75      0.82      0.78       250
           1       0.74      0.65      0.69       194

    accuracy                           0.75       444
   macro avg       0.74      0.73      0.74       444
weighted avg       0.74      0.75      0.74       444

